In [1]:
!python3 sagemaker_deployment.py \
  --role-arn arn:aws:iam::768394660366:role/datazone_usr_role_5i1vmxossv3f2o_b1rxf3f1mnfnv4 \
  --bucket lol-coach-processed-data \
  --action deploy-profiler \
  --model-data s3://lol-coach-processed-data/models/playstyle-profiler/sagemaker-scikit-learn-2025-11-05-03-06-41-484/output/model.tar.gz

sagemaker.config INFO - Fetched defaults config from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
sagemaker.config INFO - Applied value from config key = SageMaker.Model.VpcConfig
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3Bucket
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.Session.DefaultS3ObjectKeyPrefix
------!Model deployed to endpoint

In [2]:
import boto3

ENDPOINT_NAME = "playstyle-profiler-20251107-235250" # <-- ENDPOINT NAME

sm_client = boto3.client('sagemaker')

try:
    endpoint_desc = sm_client.describe_endpoint(EndpointName=ENDPOINT_NAME)
    endpoint_config_name = endpoint_desc['EndpointConfigName']
    
    config_desc = sm_client.describe_endpoint_config(EndpointConfigName=endpoint_config_name)
    model_name = config_desc['ProductionVariants'][0]['ModelName']
    
    # --- 1. Delete the Endpoint ---
    print(f"1. Deleting Endpoint: {ENDPOINT_NAME}...")
    sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
    print("   ...Waiting for endpoint to delete...")
    waiter = sm_client.get_waiter('endpoint_deleted')
    waiter.wait(EndpointName=ENDPOINT_NAME)
    print("   ...Endpoint deleted.")

    # --- 2. Delete the Endpoint Config ---
    print(f"\n2. Deleting Endpoint Config: {endpoint_config_name}...")
    sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
    print("   ...Endpoint Config deleted.")

    # --- 3. Delete the Model ---
    print(f"\n3. Deleting Model: {model_name}...")
    sm_client.delete_model(ModelName=model_name)
    print("   ...Model deleted.")
    
    print("\nAll resources successfully deleted")

except Exception as e:
    print(f"Error: {e}")
    print("Please delete resources manually in the AWS SageMaker console.")

1. Deleting Endpoint: playstyle-profiler-20251107-235250...
   ...Waiting for endpoint to delete...
   ...Endpoint deleted.

2. Deleting Endpoint Config: playstyle-profiler-20251107-235250...
   ...Endpoint Config deleted.

3. Deleting Model: sagemaker-scikit-learn-2025-11-07-23-53-03-206...
   ...Model deleted.

All resources successfully deleted
